In [1]:
# Import Dependencies
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import pandas as pd

In [2]:
# Import Csv

file_to_load = 'Resources/Mental_Health_Programs.csv'

# Convert to DF
programs_df = pd.read_csv(file_to_load)
programs_df

,X,Y,OBJECTID,source,ext_id,cat1,cat2,cat3,org_name,Name,...,zip,link,use_type,latitude,longitude,date_updated,email,dis_status,POINT_X,POINT_Y
0,-118.136071,34.695266,1,LA County Department of Mental Health,7673.0,Health and Mental Health,Mental Health Programs,NaN,NaN,Residential Program,...,93534.0,http://egis3.lacounty.gov/lms/?p=215,publish,34.695266,-118.136071,2010/11/01 19:50:56+00,NaN,NaN,6.520770e+06,2.075430e+06
1,-118.544522,34.379410,5,LA County Department of Mental Health,7413.0,Health and Mental Health,Mental Health Programs,NaN,NaN,Olive Vista,...,91321.0,http://egis3.lacounty.gov/lms/?p=541,publish,34.379410,-118.544522,2010/11/01 19:50:56+00,NaN,NaN,6.397374e+06,1.960900e+06
2,-118.486827,34.199690,14,211,NaN,Health and Mental Health,Mental Health Programs,NaN,NaN,Valley Trauma Counseling Center - Sophia Ave V...,...,91406.0,http://egis3.lacounty.gov/lms/?p=563,publish,34.199690,-118.486827,2013/06/01 11:50:56+00,NaN,NaN,6.414465e+06,1.895408e+06
3,-118.257443,34.128058,26,211,NaN,Health and Mental Health,Mental Health Programs,NaN,NaN,Glendale Memorial Hospital And Health Center,...,91204.0,http://egis3.lacounty.gov/lms/?p=570,publish,34.128058,-118.257443,2016/01/22 16:10:02+00,NaN,NaN,6.483758e+06,1.869083e+06
4,-118.265746,34.037364,55,211,NaN,Health and Mental Health,Mental Health Programs,NaN,Catholic Healthcare West,California Hospital Medical Center,...,90015.0,http://egis3.lacounty.gov/lms/?p=574,publish,34.037364,-118.265746,2016/01/21 17:29:05+00,NaN,NaN,6.481158e+06,1.836084e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,-118.203108,33.929844,65740,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,St. Francis Medical Center 3630,St. Francis Medical Center,...,90262.0,http://egis3.lacounty.gov/lms/?p=72050,publish,33.929844,-118.203108,2013/08/21 11:23:29+00,NaN,NaN,6.500056e+06,1.796911e+06
285,-118.309063,34.015125,65757,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,Pacific Clinics - Portals,Pacific Clinics/Portals,...,90052.0,http://egis3.lacounty.gov/lms/?p=72076,publish,34.015125,-118.309062,2013/09/11 08:46:40+00,NaN,NaN,6.468010e+06,1.828029e+06
286,-118.254735,33.945678,65763,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,NAMI (National Alliance for the Mentally Ill) ...,NAMI (National Alliance for the Mentally Ill) ...,...,90047.0,http://egis3.lacounty.gov/lms/?p=72117,publish,33.945678,-118.254735,2013/09/11 09:48:03+00,NaN,NaN,6.484410e+06,1.802709e+06
287,-118.207198,33.904867,65810,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,...,90221.0,http://egis3.lacounty.gov/lms/?p=72322,publish,33.904867,-118.207198,2013/08/14 08:21:34+00,NaN,NaN,6.498797e+06,1.787824e+06


In [3]:
# Examining the columns
programs_df.columns

Index(['X', 'Y', 'OBJECTID', 'source', 'ext_id', 'cat1', 'cat2', 'cat3',
       'org_name', 'Name', 'addrln1', 'addrln2', 'city', 'state', 'hours',
       'phones', 'url', 'info1', 'info2', 'post_id', 'description', 'zip',
       'link', 'use_type', 'latitude', 'longitude', 'date_updated', 'email',
       'dis_status', 'POINT_X', 'POINT_Y'],
      dtype='object')

In [4]:
# Drop Null columns
programs_df = programs_df.drop(['email', 'source', 'link', 'use_type', 'cat3', 'description', 'ext_id', 'addrln1', 'addrln2', 'info1', 'info2', 'hours', 'phones'],axis = 1)
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,url,post_id,zip,latitude,longitude,date_updated,dis_status,POINT_X,POINT_Y
0,-118.136071,34.695266,1,Health and Mental Health,Mental Health Programs,NaN,Residential Program,Lancaster,CA,NaN,215,93534.0,34.695266,-118.136071,2010/11/01 19:50:56+00,NaN,6.520770e+06,2.075430e+06
1,-118.544522,34.379410,5,Health and Mental Health,Mental Health Programs,NaN,Olive Vista,Newhall,CA,NaN,541,91321.0,34.379410,-118.544522,2010/11/01 19:50:56+00,NaN,6.397374e+06,1.960900e+06
2,-118.486827,34.199690,14,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center - Sophia Ave V...,Van Nuys,CA,www.csun.edu/vtc,563,91406.0,34.199690,-118.486827,2013/06/01 11:50:56+00,NaN,6.414465e+06,1.895408e+06
3,-118.257443,34.128058,26,Health and Mental Health,Mental Health Programs,NaN,Glendale Memorial Hospital And Health Center,Glendale,CA,http://www.glendalememorial.com/,570,91204.0,34.128058,-118.257443,2016/01/22 16:10:02+00,NaN,6.483758e+06,1.869083e+06
4,-118.265746,34.037364,55,Health and Mental Health,Mental Health Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,http://www.chmcla.org,574,90015.0,34.037364,-118.265746,2016/01/21 17:29:05+00,NaN,6.481158e+06,1.836084e+06
5,-118.186744,33.808107,82,Health and Mental Health,Mental Health Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,http://memorialcare.org/locations/search/adult...,577,90806.0,33.808107,-118.186744,2016/01/25 13:37:43+00,NaN,6.504938e+06,1.752599e+06
6,-118.588723,34.437405,96,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center - Santa Clarit...,Valencia,CA,www.csun.edu/vtc,581,91355.0,34.437405,-118.588723,2013/06/01 11:50:56+00,NaN,6.384161e+06,1.982080e+06
7,-118.307462,33.738067,108,Health and Mental Health,Mental Health Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,california.providence.org/san-pedro/Pages/defa...,591,90732.0,33.738067,-118.307462,2017/10/31 16:49:05+00,NaN,6.468187e+06,1.727199e+06
8,-118.535918,34.228586,143,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center,Northridge,CA,www.csun.edu/vtc,614,91324.0,34.228586,-118.535918,2013/06/01 11:50:56+00,NaN,6.399677e+06,1.905999e+06
9,-118.310283,33.928430,144,Health and Mental Health,Mental Health Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,www.shieldsforfamilies.org,619,90047.0,33.928430,-118.310283,2016/05/16 08:38:19+00,NaN,6.467543e+06,1.796480e+06


In [5]:
programs_df.count()

X               289
Y               289
OBJECTID        289
cat1            289
cat2            289
org_name         69
Name            289
city            288
state           288
url             214
post_id         289
zip             288
latitude        289
longitude       289
date_updated    289
dis_status        0
POINT_X         289
POINT_Y         289
dtype: int64

In [6]:
# Dropping additional columns
programs_df = programs_df.drop(['url', 'dis_status'], axis=1)
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.136071,34.695266,1,Health and Mental Health,Mental Health Programs,NaN,Residential Program,Lancaster,CA,215,93534.0,34.695266,-118.136071,2010/11/01 19:50:56+00,6.520770e+06,2.075430e+06
1,-118.544522,34.379410,5,Health and Mental Health,Mental Health Programs,NaN,Olive Vista,Newhall,CA,541,91321.0,34.379410,-118.544522,2010/11/01 19:50:56+00,6.397374e+06,1.960900e+06
2,-118.486827,34.199690,14,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center - Sophia Ave V...,Van Nuys,CA,563,91406.0,34.199690,-118.486827,2013/06/01 11:50:56+00,6.414465e+06,1.895408e+06
3,-118.257443,34.128058,26,Health and Mental Health,Mental Health Programs,NaN,Glendale Memorial Hospital And Health Center,Glendale,CA,570,91204.0,34.128058,-118.257443,2016/01/22 16:10:02+00,6.483758e+06,1.869083e+06
4,-118.265746,34.037364,55,Health and Mental Health,Mental Health Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,574,90015.0,34.037364,-118.265746,2016/01/21 17:29:05+00,6.481158e+06,1.836084e+06
5,-118.186744,33.808107,82,Health and Mental Health,Mental Health Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,577,90806.0,33.808107,-118.186744,2016/01/25 13:37:43+00,6.504938e+06,1.752599e+06
6,-118.588723,34.437405,96,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center - Santa Clarit...,Valencia,CA,581,91355.0,34.437405,-118.588723,2013/06/01 11:50:56+00,6.384161e+06,1.982080e+06
7,-118.307462,33.738067,108,Health and Mental Health,Mental Health Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,591,90732.0,33.738067,-118.307462,2017/10/31 16:49:05+00,6.468187e+06,1.727199e+06
8,-118.535918,34.228586,143,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center,Northridge,CA,614,91324.0,34.228586,-118.535918,2013/06/01 11:50:56+00,6.399677e+06,1.905999e+06
9,-118.310283,33.928430,144,Health and Mental Health,Mental Health Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,619,90047.0,33.928430,-118.310283,2016/05/16 08:38:19+00,6.467543e+06,1.796480e+06


In [7]:
# Find and drop remaining null values

programs_df.isnull().sum()

X                 0
Y                 0
OBJECTID          0
cat1              0
cat2              0
org_name        220
Name              0
city              1
state             1
post_id           0
zip               1
latitude          0
longitude         0
date_updated      0
POINT_X           0
POINT_Y           0
dtype: int64

In [8]:
# Drop rows with null values

programs_df = programs_df.dropna(subset=['state', 'zip', 'org_name'])
programs_df.isnull().sum()

X               0
Y               0
OBJECTID        0
cat1            0
cat2            0
org_name        0
Name            0
city            0
state           0
post_id         0
zip             0
latitude        0
longitude       0
date_updated    0
POINT_X         0
POINT_Y         0
dtype: int64

In [9]:
programs_df.count()
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
4,-118.265746,34.037364,55,Health and Mental Health,Mental Health Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,574,90015.0,34.037364,-118.265746,2016/01/21 17:29:05+00,6.481158e+06,1.836084e+06
5,-118.186744,33.808107,82,Health and Mental Health,Mental Health Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,577,90806.0,33.808107,-118.186744,2016/01/25 13:37:43+00,6.504938e+06,1.752599e+06
7,-118.307462,33.738067,108,Health and Mental Health,Mental Health Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,591,90732.0,33.738067,-118.307462,2017/10/31 16:49:05+00,6.468187e+06,1.727199e+06
9,-118.310283,33.928430,144,Health and Mental Health,Mental Health Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,619,90047.0,33.928430,-118.310283,2016/05/16 08:38:19+00,6.467543e+06,1.796480e+06
15,-118.259793,34.077292,199,Health and Mental Health,Mental Health Programs,www.elcentrodelpueblo.org,El Centro Del Pueblo,Los Angeles,CA,649,90026.0,34.077292,-118.259793,2013/06/01 11:50:56+00,6.483000e+06,1.850610e+06
16,-118.265498,33.918303,207,Health and Mental Health,Mental Health Programs,LA County,View Heights Convalescent Hospital,Los Angeles,CA,654,90061.0,33.918303,-118.265498,2013/05/28 15:18:27+00,6.481119e+06,1.792755e+06
17,-118.092159,34.142019,220,Health and Mental Health,Mental Health Programs,www.SGVPA.org,San Gabriel Valley Psychological Association,Pasadena,CA,662,91107.0,34.142019,-118.092159,2013/06/01 11:50:56+00,6.533784e+06,1.874077e+06
18,-118.181761,34.113639,256,Health and Mental Health,Mental Health Programs,www.dignitylosangeles.org,Dignity Los Angeles,Los Angeles,CA,678,90042.0,34.113639,-118.181761,2017/10/31 16:51:51+00,6.506653e+06,1.863786e+06
23,-117.892546,34.027694,290,Health and Mental Health,Mental Health Programs,www.almafs.com,Alma Family Services - Walnut Office,Walnut,CA,700,91789.0,34.027694,-117.892546,2013/06/01 11:50:56+00,6.594229e+06,1.832476e+06
24,-118.225448,33.923299,311,Health and Mental Health,Mental Health Programs,www.shieldsforfamilies.org,Shields For Families Inc - Eden,Lynwood,CA,712,90262.0,33.923299,-118.225448,2013/06/01 11:50:56+00,6.493274e+06,1.794544e+06


In [10]:
programs_df.count()
# Only include mental health centers listed in Los Angeles
#programs_df = centers_df.loc[centers_df['city']== 'Los Angeles']
#programs_df.head(10)


X               68
Y               68
OBJECTID        68
cat1            68
cat2            68
org_name        68
Name            68
city            68
state           68
post_id         68
zip             68
latitude        68
longitude       68
date_updated    68
POINT_X         68
POINT_Y         68
dtype: int64

In [11]:
programs_df.count()

X               68
Y               68
OBJECTID        68
cat1            68
cat2            68
org_name        68
Name            68
city            68
state           68
post_id         68
zip             68
latitude        68
longitude       68
date_updated    68
POINT_X         68
POINT_Y         68
dtype: int64

In [12]:
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
4,-118.265746,34.037364,55,Health and Mental Health,Mental Health Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,574,90015.0,34.037364,-118.265746,2016/01/21 17:29:05+00,6.481158e+06,1.836084e+06
5,-118.186744,33.808107,82,Health and Mental Health,Mental Health Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,577,90806.0,33.808107,-118.186744,2016/01/25 13:37:43+00,6.504938e+06,1.752599e+06
7,-118.307462,33.738067,108,Health and Mental Health,Mental Health Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,591,90732.0,33.738067,-118.307462,2017/10/31 16:49:05+00,6.468187e+06,1.727199e+06
9,-118.310283,33.928430,144,Health and Mental Health,Mental Health Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,619,90047.0,33.928430,-118.310283,2016/05/16 08:38:19+00,6.467543e+06,1.796480e+06
15,-118.259793,34.077292,199,Health and Mental Health,Mental Health Programs,www.elcentrodelpueblo.org,El Centro Del Pueblo,Los Angeles,CA,649,90026.0,34.077292,-118.259793,2013/06/01 11:50:56+00,6.483000e+06,1.850610e+06
16,-118.265498,33.918303,207,Health and Mental Health,Mental Health Programs,LA County,View Heights Convalescent Hospital,Los Angeles,CA,654,90061.0,33.918303,-118.265498,2013/05/28 15:18:27+00,6.481119e+06,1.792755e+06
17,-118.092159,34.142019,220,Health and Mental Health,Mental Health Programs,www.SGVPA.org,San Gabriel Valley Psychological Association,Pasadena,CA,662,91107.0,34.142019,-118.092159,2013/06/01 11:50:56+00,6.533784e+06,1.874077e+06
18,-118.181761,34.113639,256,Health and Mental Health,Mental Health Programs,www.dignitylosangeles.org,Dignity Los Angeles,Los Angeles,CA,678,90042.0,34.113639,-118.181761,2017/10/31 16:51:51+00,6.506653e+06,1.863786e+06
23,-117.892546,34.027694,290,Health and Mental Health,Mental Health Programs,www.almafs.com,Alma Family Services - Walnut Office,Walnut,CA,700,91789.0,34.027694,-117.892546,2013/06/01 11:50:56+00,6.594229e+06,1.832476e+06
24,-118.225448,33.923299,311,Health and Mental Health,Mental Health Programs,www.shieldsforfamilies.org,Shields For Families Inc - Eden,Lynwood,CA,712,90262.0,33.923299,-118.225448,2013/06/01 11:50:56+00,6.493274e+06,1.794544e+06


In [13]:
print(programs_df['city'].unique())

['Los Angeles' 'Long Beach' 'San Pedro' 'Pasadena' 'Walnut' 'Lynwood'
 'Downey' 'Sherman Oaks' 'Canoga Park' 'Venice' 'Hawaiian Gardens'
 'Inglewood' 'Redondo Beach' 'Santa Monica' 'San Gabriel' 'Northridge'
 'Baldwin Park' 'Pacoima' 'Lancaster' 'Burbank' 'Arcadia' 'Los Angeles '
 'Compton' 'Signal Hill ' 'Los Angelels']


In [14]:
# Find remaining null values
programs_df.isnull().sum()

X               0
Y               0
OBJECTID        0
cat1            0
cat2            0
org_name        0
Name            0
city            0
state           0
post_id         0
zip             0
latitude        0
longitude       0
date_updated    0
POINT_X         0
POINT_Y         0
dtype: int64

In [15]:
# Drop remaining null values
programs_df = programs_df.dropna(subset=['state', 'zip','city'])
programs_df.isnull().sum()

X               0
Y               0
OBJECTID        0
cat1            0
cat2            0
org_name        0
Name            0
city            0
state           0
post_id         0
zip             0
latitude        0
longitude       0
date_updated    0
POINT_X         0
POINT_Y         0
dtype: int64

In [16]:
# Sort the df by Zip code

programs_sorted = programs_df.sort_values(by = 'zip', ascending = False)
programs_sorted

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
105,-118.146436,34.688680,1527,Health and Mental Health,Mental Health Programs,www.avysa.org,Helplines Of Youth Support Association,Lancaster,CA,1347,93534.0,34.688680,-118.146435,2013/06/01 11:50:56+00,6.517651e+06,2.073038e+06
23,-117.892546,34.027694,290,Health and Mental Health,Mental Health Programs,www.almafs.com,Alma Family Services - Walnut Office,Walnut,CA,700,91789.0,34.027694,-117.892546,2013/06/01 11:50:56+00,6.594229e+06,1.832476e+06
73,-118.100244,34.097646,893,Health and Mental Health,Mental Health Programs,www.asianyouthcenter.org,Asian Youth Center,San Gabriel,CA,1061,91776.0,34.097646,-118.100244,2016/05/02 12:40:17+00,6.531322e+06,1.857931e+06
78,-117.964652,34.087127,915,Health and Mental Health,Mental Health Programs,www.baldwinpark.com,City Of Baldwin Park Department Of Recreation ...,Baldwin Park,CA,1068,91706.0,34.087127,-117.964652,2017/10/30 14:25:53+00,6.572373e+06,1.854089e+06
107,-118.306259,34.181243,1551,Health and Mental Health,Mental Health Programs,www.salvationarmy-socal.org,Salvation Army Corps Community Center - Burbank,Burbank,CA,1360,91502.0,34.181243,-118.306259,2013/06/01 11:50:56+00,6.469044e+06,1.888479e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,-118.273613,34.021224,61877,Health and Mental Health,Mental Health Programs,Hill Street Community Wellness Center,Hill Street Community Wellness Center,Los Angeles,CA,68346,90007.0,34.021224,-118.273613,2013/08/21 16:07:55+00,6.478759e+06,1.830217e+06
113,-118.303030,34.062952,55734,Health and Mental Health,Mental Health Programs,AIDS Project Los Angeles (APLA),AIDS Project Los Angeles (APLA),Los Angeles,CA,57722,90005.0,34.062952,-118.303030,2013/10/24 14:53:28+00,6.469891e+06,1.845428e+06
49,-118.264985,33.948993,525,Health and Mental Health,Mental Health Programs,www.resourcesunlimited.us/,Los Angeles Wings Of Faith Inc,Los Angeles,CA,841,90003.0,33.948993,-118.264985,2013/06/01 11:50:56+00,6.481304e+06,1.803924e+06
44,-118.244546,33.953187,459,Health and Mental Health,Mental Health Programs,www.drewcdc.org,92Nd And Maie Child Development Center,Los Angeles,CA,804,90002.0,33.953187,-118.244546,2017/10/16 14:16:17+00,6.487507e+06,1.805434e+06


In [17]:
programs_sorted.count()

X               68
Y               68
OBJECTID        68
cat1            68
cat2            68
org_name        68
Name            68
city            68
state           68
post_id         68
zip             68
latitude        68
longitude       68
date_updated    68
POINT_X         68
POINT_Y         68
dtype: int64

In [18]:
programs_sorted['Program_Type'] = 'Substance Abuse Program'
programs_sorted

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y,Program_Type
105,-118.146436,34.688680,1527,Health and Mental Health,Mental Health Programs,www.avysa.org,Helplines Of Youth Support Association,Lancaster,CA,1347,93534.0,34.688680,-118.146435,2013/06/01 11:50:56+00,6.517651e+06,2.073038e+06,Substance Abuse Program
23,-117.892546,34.027694,290,Health and Mental Health,Mental Health Programs,www.almafs.com,Alma Family Services - Walnut Office,Walnut,CA,700,91789.0,34.027694,-117.892546,2013/06/01 11:50:56+00,6.594229e+06,1.832476e+06,Substance Abuse Program
73,-118.100244,34.097646,893,Health and Mental Health,Mental Health Programs,www.asianyouthcenter.org,Asian Youth Center,San Gabriel,CA,1061,91776.0,34.097646,-118.100244,2016/05/02 12:40:17+00,6.531322e+06,1.857931e+06,Substance Abuse Program
78,-117.964652,34.087127,915,Health and Mental Health,Mental Health Programs,www.baldwinpark.com,City Of Baldwin Park Department Of Recreation ...,Baldwin Park,CA,1068,91706.0,34.087127,-117.964652,2017/10/30 14:25:53+00,6.572373e+06,1.854089e+06,Substance Abuse Program
107,-118.306259,34.181243,1551,Health and Mental Health,Mental Health Programs,www.salvationarmy-socal.org,Salvation Army Corps Community Center - Burbank,Burbank,CA,1360,91502.0,34.181243,-118.306259,2013/06/01 11:50:56+00,6.469044e+06,1.888479e+06,Substance Abuse Program
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,-118.273613,34.021224,61877,Health and Mental Health,Mental Health Programs,Hill Street Community Wellness Center,Hill Street Community Wellness Center,Los Angeles,CA,68346,90007.0,34.021224,-118.273613,2013/08/21 16:07:55+00,6.478759e+06,1.830217e+06,Substance Abuse Program
113,-118.303030,34.062952,55734,Health and Mental Health,Mental Health Programs,AIDS Project Los Angeles (APLA),AIDS Project Los Angeles (APLA),Los Angeles,CA,57722,90005.0,34.062952,-118.303030,2013/10/24 14:53:28+00,6.469891e+06,1.845428e+06,Substance Abuse Program
49,-118.264985,33.948993,525,Health and Mental Health,Mental Health Programs,www.resourcesunlimited.us/,Los Angeles Wings Of Faith Inc,Los Angeles,CA,841,90003.0,33.948993,-118.264985,2013/06/01 11:50:56+00,6.481304e+06,1.803924e+06,Substance Abuse Program
44,-118.244546,33.953187,459,Health and Mental Health,Mental Health Programs,www.drewcdc.org,92Nd And Maie Child Development Center,Los Angeles,CA,804,90002.0,33.953187,-118.244546,2017/10/16 14:16:17+00,6.487507e+06,1.805434e+06,Substance Abuse Program


In [19]:
# Export Sorted Df to csv in Cleaned_Data folder

programs_sorted.to_csv('Cleaned_Data/mh_Substance_Abuse_Programs.csv')